In [1]:
!nvidia-smi

Thu Jun 26 00:53:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
!unzip -q /content/data_labeled.zip -d /content/.

In [2]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

def split_img_label(data_train, data_test, folder_train, folder_test):
    os.makedirs(folder_train + '/images', exist_ok=True)
    os.makedirs(folder_train + '/labels', exist_ok=True)
    os.makedirs(folder_test + '/images', exist_ok=True)
    os.makedirs(folder_test + '/labels', exist_ok=True)

    for img_path in data_train:
        img_name = os.path.basename(img_path)
        label_name = img_name.replace('.jpg', '.txt')

        shutil.copy(img_path, f"{folder_train}/images/{img_name}")
        label_path = os.path.join(PATH_labels, label_name)
        if os.path.exists(label_path):
            shutil.copy(label_path, f"{folder_train}/labels/{label_name}")

    for img_path in data_test:
        img_name = os.path.basename(img_path)
        label_name = img_name.replace('.jpg', '.txt')

        shutil.copy(img_path, f"{folder_test}/images/{img_name}")
        label_path = os.path.join(PATH_labels, label_name)
        if os.path.exists(label_path):
            shutil.copy(label_path, f"{folder_test}/labels/{label_name}")

In [3]:
PATH_images = '/content/data_labeled/images/'
PATH_labels = '/content/data_labeled/labels/'

list_img = [img for img in os.listdir(PATH_images) if img.endswith('.jpg')]
path_img = [os.path.join(PATH_images, img) for img in list_img]

df = pd.DataFrame(path_img)
data_train, data_test, _, _ = train_test_split(df[0], df.index, test_size=0.20, random_state=42)

Train = '/content/data/train'
Test = '/content/data/val'

split_img_label(data_train, data_test, Train, Test)

In [4]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [5]:
import yaml

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'val/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/data_labeled/classes.txt'
path_to_data_yaml = '/content/data/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data/data.yaml

Created config file at /content/data/data.yaml

File contents:

path: /content/data
train: train/images
val: val/images
nc: 1
names:
- staff


In [6]:
!yolo detect train data=/content/data/data.yaml model=yolo11s.pt epochs=40 imgsz=960

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
100% 18.4M/18.4M [00:00<00:00, 439MB/s]
Ultralytics 8.3.159 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=960, int8=Fal

In [7]:
!mkdir /content/custom_model
!cp /content/runs/detect/train/weights/best.pt /content/custom_model/custom_model.pt
!cp -r /content/runs/detect/train /content/custom_model

%cd custom_model
!zip /content/custom_model.zip custom_model.pt
!zip -r /content/custom_model.zip train
%cd /content

/content/custom_model
  adding: custom_model.pt (deflated 9%)
  adding: train/ (stored 0%)
  adding: train/train_batch1.jpg (deflated 2%)
  adding: train/F1_curve.png (deflated 17%)
  adding: train/weights/ (stored 0%)
  adding: train/weights/best.pt (deflated 9%)
  adding: train/weights/last.pt (deflated 9%)
  adding: train/labels.jpg (deflated 44%)
  adding: train/train_batch210.jpg (deflated 3%)
  adding: train/train_batch212.jpg (deflated 6%)
  adding: train/val_batch0_pred.jpg (deflated 3%)
  adding: train/P_curve.png (deflated 23%)
  adding: train/train_batch211.jpg (deflated 5%)
  adding: train/val_batch0_labels.jpg (deflated 3%)
  adding: train/PR_curve.png (deflated 30%)
  adding: train/results.csv (deflated 62%)
  adding: train/labels_correlogram.jpg (deflated 48%)
  adding: train/train_batch0.jpg (deflated 2%)
  adding: train/args.yaml (deflated 53%)
  adding: train/confusion_matrix.png (deflated 40%)
  adding: train/confusion_matrix_normalized.png (deflated 39%)
  adding: t

In [8]:
!zip -r /content/data.zip data

  adding: data/ (stored 0%)
  adding: data/train/ (stored 0%)
  adding: data/train/labels.cache (deflated 84%)
  adding: data/train/labels/ (stored 0%)
  adding: data/train/labels/305d8b3a-dataframe383.txt (deflated 27%)
  adding: data/train/labels/db034151-dataframe1017.txt (deflated 27%)
  adding: data/train/labels/07fd53dc-dataframe382.txt (deflated 27%)
  adding: data/train/labels/99c04c61-dataframe1199.txt (deflated 26%)
  adding: data/train/labels/53587b05-dataframe1058.txt (deflated 28%)
  adding: data/train/labels/67d9c123-dataframe1114.txt (deflated 27%)
  adding: data/train/labels/f9990d64-dataframe1205.txt (deflated 24%)
  adding: data/train/labels/ab715818-dataframe810.txt (deflated 27%)
  adding: data/train/labels/f8541eef-dataframe1155.txt (deflated 26%)
  adding: data/train/labels/11a46f2c-dataframe1059.txt (deflated 26%)
  adding: data/train/labels/fa6cd1e5-dataframe1020.txt (deflated 25%)
  adding: data/train/labels/27d20624-dataframe1117.txt (deflated 27%)
  adding: d